Source:
    https://medium.com/@connectwithghosh/recommender-system-on-the-movielens-using-an-autoencoder-using-tensorflow-in-python-f13d3e8d600d

In [1]:
# Importing tensorflow
import tensorflow as tf
# Importing some more libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

In [12]:
# reading the ratings data
# contains UserID::MovieID::Rating::Timestamp
ratings = pd.read_csv('../datasets/ml-1m/ratings.dat',\
          sep="::", header = None, engine='python')

ratings.head(2)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109


### Selecting some features

UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2) (dropped)
- Each user has at least 20 ratings

In [16]:
# Lets pivot the data to get it at a user level
# To generate a table with userXmovies
# 0)UserID::1)MovieID::2)Rating::3)Timestamp
ratings_pivot = pd.pivot_table(ratings[[0,1,2]],\
          values=2, index=0, columns=1 ).fillna(0)

ratings_pivot.head(2)


1,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
ratings_pivot.shape

(6040, 3706)

In [18]:
# creating train and test sets
X_train, X_test = train_test_split(ratings_pivot, train_size=0.8)

In [28]:
X_train.shape[0]
display("trainset:rows {} and cols {}".format(X_train.shape[0],X_train.shape[1]))
display("testset:rows {} and cols {}".format(X_test.shape[0],X_test.shape[1]))

'trainset:rows 4832 and cols 3706'

'testset:rows 1208 and cols 3706'

In [31]:
# Deciding how many nodes wach layer should have
n_nodes_inpl = 3706
n_nodes_hl1  = 256
n_nodes_outl = 3706  

hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_inpl+1,n_nodes_hl1]))}

output_layer_vals = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1+1,n_nodes_outl])) }

**NOTE:** The bias matrix is not created, due to was added a bias node to each layer which has a contant value = 1

In [32]:
# user with 3706 ratings goes in
input_layer = tf.placeholder('float', [None, 3706])
# add a constant node to the first layer
# it needs to have the same shape as the input layer for me to be
# able to concatinate it later
input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)
# multiply output of input_layer wth a weight matrix 
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat,hidden_1_layer_vals['weights']))
# adding one bias node to the hidden layer
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)
# multiply output of hidden with a weight matrix to get final output
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])
# output_true shall have the original shape for error calculations
output_true = tf.placeholder('float', [None, 3706])
# define our cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
# define our optimizer
learn_rate = 0.1   # how fast the model should learn
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

W0916 20:59:56.061133 140424641423168 deprecation.py:506] From /home/i2t/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/adagrad.py:76: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [35]:
# initialising variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# defining batch size, number of epochs and learning rate
batch_size = 100  # how many images to use together for training
hm_epochs = 200    # how many times to go through the entire dataset
tot_users = X_train.shape[0] # total number of users

In [36]:
# running the model for a 200 epochs taking 100 users in batches
# total improvement is printed out after each epoch
for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing error as 0
    
    for i in range(int(tot_users/batch_size)):
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],\
               feed_dict={input_layer: epoch_x, \
               output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer, feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer, feed_dict={input_layer:X_test})
        
    print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))      
    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)

MSE train 57.39390116577901 MSE test 57.38934683141875
Epoch 0 / 200 loss: 3900.6919326782227
MSE train 38.74890892495517 MSE test 38.6264581985595
Epoch 1 / 200 loss: 2272.209987640381
MSE train 28.57256802106744 MSE test 28.33049441343024
Epoch 2 / 200 loss: 1606.6325798034668
MSE train 22.056296315056844 MSE test 21.801614362521907
Epoch 3 / 200 loss: 1212.1367206573486
MSE train 17.966431810009347 MSE test 17.72979923083822
Epoch 4 / 200 loss: 961.5949630737305
MSE train 15.382106282760871 MSE test 15.197460855405028
Epoch 5 / 200 loss: 802.8688144683838
MSE train 13.533191625399228 MSE test 13.427552588569121
Epoch 6 / 200 loss: 697.210147857666
MSE train 12.144757350662331 MSE test 12.108740681297617
Epoch 7 / 200 loss: 619.8052282333374
MSE train 11.069811636424141 MSE test 11.086406935737093
Epoch 8 / 200 loss: 560.4802780151367
MSE train 10.20652292856114 MSE test 10.26344200669845
Epoch 9 / 200 loss: 513.6228876113892
MSE train 9.506388863110384 MSE test 9.567601012902006
Epo

MSE train 2.6381636272929683 MSE test 3.0690478000402153
Epoch 87 / 200 loss: 126.70005941390991
MSE train 2.6233239792878487 MSE test 3.05440022622542
Epoch 88 / 200 loss: 125.96895611286163
MSE train 2.608837936670228 MSE test 3.0399516017520294
Epoch 89 / 200 loss: 125.25940680503845
MSE train 2.5944549122176386 MSE test 3.025760806645409
Epoch 90 / 200 loss: 124.56600201129913
MSE train 2.579676499130881 MSE test 3.0115063328656797
Epoch 91 / 200 loss: 123.87492084503174
MSE train 2.5645282613525793 MSE test 2.997226817896652
Epoch 92 / 200 loss: 123.16407585144043
MSE train 2.5494610748676387 MSE test 2.9831148156164433
Epoch 93 / 200 loss: 122.43786382675171
MSE train 2.534651545489577 MSE test 2.9692299045062596
Epoch 94 / 200 loss: 121.71763062477112
MSE train 2.5201742413120165 MSE test 2.9556650710153405
Epoch 95 / 200 loss: 121.0106920003891
MSE train 2.5058281535574887 MSE test 2.9422867971982583
Epoch 96 / 200 loss: 120.31783390045166
MSE train 2.4916525398096647 MSE test 

MSE train 1.8948630031665794 MSE test 2.3774756663464682
Epoch 173 / 200 loss: 90.65025770664215
MSE train 1.8901964458979623 MSE test 2.3731846931636627
Epoch 174 / 200 loss: 90.42403721809387
MSE train 1.885539147168526 MSE test 2.3688928384001353
Epoch 175 / 200 loss: 90.20000171661377
MSE train 1.8808472959333904 MSE test 2.364582982481707
Epoch 176 / 200 loss: 89.9766274690628
MSE train 1.8761525408280908 MSE test 2.3602799422045617
Epoch 177 / 200 loss: 89.75187695026398
MSE train 1.8714094650357418 MSE test 2.356025854620243
Epoch 178 / 200 loss: 89.52642953395844
MSE train 1.8666026543385732 MSE test 2.351771694841121
Epoch 179 / 200 loss: 89.29800391197205
MSE train 1.8618545174061423 MSE test 2.347574119248909
Epoch 180 / 200 loss: 89.06757509708405
MSE train 1.8570167769165877 MSE test 2.343409053591541
Epoch 181 / 200 loss: 88.84009897708893
MSE train 1.8520951184636727 MSE test 2.3392625073361226
Epoch 182 / 200 loss: 88.60793697834015
MSE train 1.8470857857233438 MSE test

In [37]:
# pick a user
sample_user = X_test.iloc[99,:]
#get the predicted ratings
sample_user_pred = sess.run(output_layer, feed_dict={input_layer:[sample_user]})

In [44]:
display(sample_user.shape)
sample_user

(3706,)

1
1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
6       0.0
7       0.0
8       0.0
9       0.0
10      0.0
11      3.0
12      0.0
13      0.0
14      0.0
15      0.0
16      0.0
17      0.0
18      0.0
19      0.0
20      0.0
21      0.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      0.0
28      0.0
29      0.0
30      0.0
       ... 
3923    0.0
3924    0.0
3925    0.0
3926    0.0
3927    0.0
3928    0.0
3929    0.0
3930    0.0
3931    0.0
3932    0.0
3933    0.0
3934    0.0
3935    0.0
3936    0.0
3937    0.0
3938    0.0
3939    0.0
3940    0.0
3941    0.0
3942    0.0
3943    0.0
3944    0.0
3945    0.0
3946    0.0
3947    0.0
3948    0.0
3949    0.0
3950    0.0
3951    0.0
3952    0.0
Name: 4792, Length: 3706, dtype: float64

In [46]:
display(sample_user_pred.shape)
sample_user_pred

(1, 3706)

array([[-1.7229145 ,  0.9290279 ,  0.06855589, ...,  1.3700454 ,
        -0.06047785, -0.4500605 ]], dtype=float32)